In [1]:
import os
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix, vstack, load_npz

import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
from tqdm.notebook import tqdm
from sys import getsizeof
import gc
#from catboost import CatBoostRegressor, cv, Pool, sum_models
from sklearn.model_selection import StratifiedKFold, train_test_split
#from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.preprocessing import MaxAbsScaler
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDClassifier
from catboost import CatBoostClassifier, CatBoostRegressor, cv, Pool, sum_models, EShapCalcType, EFeaturesSelectionAlgorithm

import vaex
import pyarrow.parquet as pq
import bisect

import pickle

C:\ProgramData\Miniconda3\envs\ml_env_v1\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Miniconda3\envs\ml_env_v1\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\ProgramData\Miniconda3\envs\ml_env_v1\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
C:\ProgramData\Miniconda3\envs\ml_env_v1\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
LOCAL_DATA_PATH = 'context_data'
SPLIT_SEED = 42
DATA_FILE = 'competition_data_final_pqt'
TARGET_FILE = 'public_train.pqt'
SUBMISSION_FILE = 'submit_2.pqt'

In [3]:
id_to_submit = pq.read_table(f'../{LOCAL_DATA_PATH}/{SUBMISSION_FILE}').to_pandas()
tgt = pq.read_table(f'../{LOCAL_DATA_PATH}/{TARGET_FILE}').to_pandas()

In [4]:
mat = load_npz('mat.npz')
mat

<415317x199683 sparse matrix of type '<class 'numpy.uint16'>'
	with 32277669 stored elements in Compressed Sparse Row format>

In [5]:
sc = MaxAbsScaler()
mat = sc.fit_transform(mat)

In [35]:
%%time
Ny = Nystroem(kernel='laplacian', n_components=1000, random_state=722, n_jobs=-1)
emb = Ny.fit_transform(mat)

CPU times: total: 6min 43s
Wall time: 28.3 s


In [36]:
emb

array([[0.03161442, 0.03159824, 0.03159444, ..., 0.03163573, 0.03157541,
        0.03166035],
       [0.03164279, 0.03162547, 0.03164188, ..., 0.03162936, 0.03155789,
        0.03164372],
       [0.03160831, 0.03162306, 0.03161042, ..., 0.03161816, 0.03158432,
        0.03160646],
       ...,
       [0.03161288, 0.03160652, 0.03159456, ..., 0.03162352, 0.03156314,
        0.03165066],
       [0.03161295, 0.03160652, 0.03159457, ..., 0.03162343, 0.03156313,
        0.03165012],
       [0.03161301, 0.03160652, 0.03159455, ..., 0.03162346, 0.03156314,
        0.03165018]])

In [37]:
with open('nys/emb.pickle', 'wb') as outp:
    pickle.dump(emb, outp, pickle.HIGHEST_PROTOCOL)

In [32]:
idx_tr = tgt[(tgt.is_male == '0') | (tgt.is_male == '1')].index.values
X = emb[idx_tr]
y = tgt[(tgt.is_male == '0') | (tgt.is_male == '1')].is_male.values

In [36]:
%%time


models_1 = []
params = {'iterations': 12000,
          'depth': 5,
          #'random_strength': 2.5,
          #'l2_leaf_reg': 3,
          'learning_rate': 0.1,
          'border_count': 256,
          #'grow_policy': 'Lossguide',
          #'loss_function': FocalLossObjective(),
          #'custom_metric': ['AUC'],
          #'eval_metric': "AUC", 
          'score_function': 'NewtonCosine',
          'task_type': "GPU",
          'thread_count': 16,
          'early_stopping_rounds': 500,
          'verbose': 2000, # output to stdout info about training process every 200 iterations
          'random_seed': 722
         }

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
for train_idx, test_idx in cv.split(X, y):
    cbc = CatBoostClassifier(**params)
    cbc.fit(X[train_idx], y[train_idx], 
        eval_set=(X[test_idx], y[test_idx]), # data to validate on
        use_best_model=True, 
        plot=True 
        )
    models_1.append(cbc)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6866517	test: 0.6865818	best: 0.6865818 (0)	total: 82.2ms	remaining: 16m 26s
2000:	learn: 0.4240968	test: 0.4953351	best: 0.4953209 (1997)	total: 44s	remaining: 3m 39s
4000:	learn: 0.3586252	test: 0.4915718	best: 0.4915554 (3998)	total: 1m 27s	remaining: 2m 55s
bestTest = 0.4909954383
bestIteration = 4597
Shrink model to first 4598 iterations.


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6873683	test: 0.6874712	best: 0.6874712 (0)	total: 90ms	remaining: 17m 59s
2000:	learn: 0.4239585	test: 0.4958862	best: 0.4958862 (2000)	total: 44.5s	remaining: 3m 42s
bestTest = 0.4923273757
bestIteration = 3250
Shrink model to first 3251 iterations.


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6873077	test: 0.6875480	best: 0.6875480 (0)	total: 77.9ms	remaining: 15m 35s
2000:	learn: 0.4235196	test: 0.4978607	best: 0.4978607 (2000)	total: 43.8s	remaining: 3m 39s
4000:	learn: 0.3578791	test: 0.4942126	best: 0.4942062 (3879)	total: 1m 27s	remaining: 2m 55s
bestTest = 0.4937623767
bestIteration = 4685
Shrink model to first 4686 iterations.


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6868688	test: 0.6869474	best: 0.6869474 (0)	total: 91.5ms	remaining: 18m 17s
2000:	learn: 0.4241697	test: 0.4971563	best: 0.4971441 (1999)	total: 43.8s	remaining: 3m 38s
4000:	learn: 0.3586775	test: 0.4928937	best: 0.4928565 (3991)	total: 1m 27s	remaining: 2m 55s
bestTest = 0.4925080763
bestIteration = 5053
Shrink model to first 5054 iterations.


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6868452	test: 0.6870977	best: 0.6870977 (0)	total: 84.7ms	remaining: 16m 56s
2000:	learn: 0.4239064	test: 0.4956504	best: 0.4956470 (1998)	total: 44s	remaining: 3m 39s
4000:	learn: 0.3586415	test: 0.4920601	best: 0.4920354 (3968)	total: 1m 28s	remaining: 2m 56s
bestTest = 0.4919422926
bestIteration = 4131
Shrink model to first 4132 iterations.
CPU times: total: 24min 30s
Wall time: 13min 19s
